In [26]:
from finefood import preprocessing, utils
from keras.utils import np_utils
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import h5py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import string
import numpy as np
from finefood import preprocessing, utils

In [3]:
path = "./data/Reviews.csv"

In [4]:
df = pd.read_csv(path).set_index("Id")

In [10]:
stemmer = SnowballStemmer('english')

def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [11]:
clean_texts = df['Text'].map(lambda x: clean_text(x))

In [12]:
clean_texts

Id
1         bought sever vital can dog food product found ...
2         product arriv label jumbo salt peanut the pean...
3         confect around centuri light pillowi citrus ge...
4         look secret ingredi robitussin believ found it...
5         great taffi great price wide assort yummi taff...
6         got wild hair taffi order five pound bag taffi...
7         saltwat taffi great flavor soft chewi candi in...
8         taffi good soft chewi flavor amaz would defini...
9         right i am most sprout cat eat grass love it r...
10        healthi dog food good digest also good small p...
11        know cactus tequila uniqu combin ingredi flavo...
12        one boy need lose weight did not put food floo...
13        cat happili eat felida platinum two year got n...
14        good flavor ! came secur pack fresh delici ! l...
15        strawberri twizzler guilti pleasur yummi six p...
16        daughter love twizzler shipment six pound real...
17        love eat good watch look mo

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [15]:
MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(clean_texts)
sequences = tokenizer.texts_to_sequences(clean_texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(df.Score))[:, 1:]
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Found 92578 unique tokens.


NameError: name 'MAX_SEQUENCE_LENGTH' is not defined

In [20]:
corpus, word_to_index, word_to_vec_map = utils.read_glove_vecs("./data/glove.6B.100d.txt")

In [23]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((MAX_NUM_WORDS, 100))
for word, index in tokenizer.word_index.items():
    if index > MAX_NUM_WORDS - 1:
        break
    else:
        embedding_vector = word_to_vec_map.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [24]:
embedding_matrix.shape

(20000, 100)

In [29]:
with h5py.File("./data/Reviews.h5", "w") as fp:
    fp.create_dataset("text_indices", data=data,  compression="gzip")
    fp["scores_oh"] = labels
    fp.create_dataset("embedding_matrix", data=embedding_matrix,  compression="gzip")
    

In [32]:
from collections import Counter

In [34]:
Counter(df.Score.values)

Counter({5: 363122, 1: 52268, 4: 80655, 2: 29769, 3: 42640})

In [35]:
df.Score.count()

568454

In [36]:
363122/568454

0.6387887146541321